
**Created by:**

__Viktor Varga__

<br>

<img src="https://docs.google.com/uc?export=download&id=1WzgXsCoz8O-NeBlJTbuLPC1iIFDmgYt1" style="display:inline-block">
<hr>

# Python tutorial - Numpy, haladó szintű `ndarray` shape manipuláció, a nézetek működésének részletei

## `ndarray.strides`

Amikor létrehozunk egy Numpy tömböt (például az `np.array()`, `np.zeros()`, `np.ones()`, `np.random()`, stb. segítségével), a memóriában egy összefüggő terület foglalódik le. Függetlenül attól, hogy egy tömb hány dimenziós alakot ölt, a memóriaterület címzése szekvenciális, nem többdimenziós.

Az `ndarray` típus megvalósítása olyan, hogy látszólag többdimenziós tömbként értelmezhetjük ezt a memóriaterületet, vagy ennek egy részét. A többdimenziós koordináták és memóriacímek közti konverziót a megvalósítás elfedi, így nekünk ezzel nem szükséges foglalkozni. Mégis, az `ndarray.shape` és `ndarray.strides` attribútumok viselkedésének tisztázásával jobban megérthető a tömbök és nézetek működése is.

Készítsünk egy kétdimenziós `np.int32` típusú tömböt! Nézzük meg, hogy mi a tömb alakja (shape), a lépésközei (strides), illetve elemeinek memóriában elfoglalt mérete (itemsize)!

In [ ]:
import numpy as np

a = np.arange(6, dtype=np.int32).reshape((2,3))

print("The array 'a':\n", a)
print("  ... its shape:", a.shape)
print("  ... its strides:", a.strides)
print("  ... size of its items (in byte):", a.itemsize)

The array 'a':
 [[0 1 2]
 [3 4 5]]
  ... its shape: (2, 3)
  ... its strides: (12, 4)
  ... size of its items (in byte): 4


Az `a` tömb alakja (2,3), lépésközei (12,4), valamint elemei egyenként 4 bájtot foglalnak.

A `strides` attribútum azt adja meg, hogy ahhoz, hogy az egyes tengelyek mentén az indexet eggyel megnöveljük, a memóriában hány bájtot kell lépnünk. Például, a (2,3) alakú 4 bájtos elemmérettel rendelkező tömb `a[0,0]` elemétől indulva `a[0,1]` eléréséhez 4 bájtot, `a[1,0]` elem eléréséhez 12 bájtot, míg `a[1,1]` eléréséhez 12+4 = 16 bájtot kell a memóriában lépni. Ha ezeket az értékeket leosztjuk az elemek méretével (int32 = 32 bit = 4 bájt), akkor azt kapjuk, hogy a #0 indexű tengely mentén egy lépés a memóriában 1 elemnyi, míg az #1 indexű tengely mentén egy lépés a memóriában 3 elemnyi méretű. Az alábbi ábra ezt szemlélteti:

![alt text](https://drive.google.com/uc?export=download&id=1KI_yNqvy038yO7lmHeiK9DpJzLSJ-tRE)

Általánosságban tehát az `a[0,0,0,...]` elemhez képest az `a[idx0, idx1, idx2, ...]` elem eléréséhez `a.itemsize * (idx0*a.strides[0] + idx1*a.strides[1] + idx2*a.strides[2] + ...)` bájtot kell lépni a memóriában.


## A tömb alakjának megváltoztatása

Nézzük meg, hogy egy 12 elemű int32 típusú tömb alakjának változtatása milyen hatással van a lépésközökre!

In [ ]:
a = np.arange(12, dtype=np.int32).reshape((4,3))

print("The 2D array 'a':\n", a)
print("  ... its shape:", a.shape)
print("  ... its strides:", a.strides)

a = a.reshape((-1))
print("\nThe flattened array 'a':\n", a)
print("  ... its shape:", a.shape)
print("  ... its strides:", a.strides)

a = a.reshape((2,2,3))
print("\nThe 3D array 'a':\n", a)
print("  ... its shape:", a.shape)
print("  ... its strides:", a.strides)




The 2D array 'a':
 [[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]]
  ... its shape: (4, 3)
  ... its strides: (12, 4)

The flattened array 'a':
 [ 0  1  2  3  4  5  6  7  8  9 10 11]
  ... its shape: (12,)
  ... its strides: (4,)

The 3D array 'a':
 [[[ 0  1  2]
  [ 3  4  5]]

 [[ 6  7  8]
  [ 9 10 11]]]
  ... its shape: (2, 2, 3)
  ... its strides: (24, 12, 4)


Semmi váratlant nem tapasztaltunk: a kilapított, 1 dimenzióssá alkított tömb lépésköze (4,), tehát az egyetlen tengely mentén 4-4 bájtot, azaz egy-egy elemet kell a memóriában lépni az index növeléséhez.

A 12 elemű tömböt 3 dimenzióssá, (2,2,3) alakúvá formázva, az első tengely mentén a lépésköz még nagyobb, 24 bájtos, azaz 6 elem hosszú.

## Basic indexing

A basic indexing lényege, hogy a tömb több elemét hivatkozzuk egyszerre, méghozzá szabályos lépésközök szerint. Ha basic indexing-el készítünk egy új nézet tömböt egy másik tömbből, akkor a nézet tömb típusa is `ndarray`, hasonlóan az eredeti tömb típusához, tehát első ránézésre megkülönböztethetetlen egy nézet és egy eredeti tömb. Azonban a nézet által mutatott memóriaterület nem feltétlenül folytonos, illetve nem feltétlenül a teljes, tömbhöz lefoglalt memóriaterületet fedi le.

Nézzük meg hogyan változik a lépésköz, ha egy 2 dimenziós tömb egyik oszlopából csinálunk egy új nézetet.

In [ ]:
a = np.arange(6, dtype=np.int32).reshape((2,3))
print("The 2D array 'a':\n", a)
print("  ... its shape:", a.shape)
print("  ... its strides:", a.strides)

b = a[:,1]
print("\nThe view 'b' of array 'a':\n", b)
print("  ... its shape:", b.shape)
print("  ... its strides:", b.strides)

The 2D array 'a':
 [[0 1 2]
 [3 4 5]]
  ... its shape: (2, 3)
  ... its strides: (12, 4)

The view 'b' of array 'a':
 [1 4]
  ... its shape: (2,)
  ... its strides: (12,)


Mint látjuk, a `b` nézet elemmérete hiába 4 bájt, a `b` tömbön való iterációhoz 12 bájtokat kell ugrálnunk. Valójában tehát a nézet az eredeti memóriaterület egy részletére mutat, de megváltoztatott `shape` és `strides` attribútumokkal, így az iteráció is megváltozik.

Nézzünk egy kicsit bonyolultabb példát basic indexing-re!

In [ ]:
a = np.arange(6, dtype=np.int32).reshape((2,3))
print("The 2D array 'a':\n", a)
print("  ... its shape:", a.shape)
print("  ... its strides:", a.strides)

b = a[1::-2]
print("\nThe view 'b':\n", b)
print("  ... its shape:", b.shape)
print("  ... its strides:", b.strides)

The 2D array 'a':
 [[0 1 2]
 [3 4 5]]
  ... its shape: (2, 3)
  ... its strides: (12, 4)

The view 'b':
 [[3 4 5]]
  ... its shape: (1, 3)
  ... its strides: (-24, 4)


Negatív lépésközzel való indexelés esetén a `strides` a megfelelő tengely mentén negatív lesz.

Bár erre a gyakorlatban nincs szükség, de egy "eredeti" tömböt (melynek létrehozásakor a mutatott memóriaterület eredetileg lefoglalódott), illetve egy nézetet, az `ndarray.base` attribútum vizsgálatával különböztethetünk meg. Ez az attribútum egy nézet esetén megadja az "eredeti tömböt", míg egy "eredeti tömb" esetén `None` az értéke.

In [ ]:
a = np.arange(6, dtype=np.int32)
print("The base of the original 'a' array:", a.base)  # None, as 'a' is an original array

b = a.reshape((2,3))
print("The base of the reshaped 'b' view:", b.base)   # not None, since 'b' is a view

c = b[:,1]
print("The base of the view 'c':", c.base)   # not None, since 'c' is a view

The base of the original 'a' array: None
The base of the reshaped 'b' view: [0 1 2 3 4 5]
The base of the view 'c': [0 1 2 3 4 5]


A fenti példákból könnyen megérthető, hogy miért készül nézet basic indexing esetén és miért szükséges másolat advanced indexing esetén. Mivel basic indexing esetén szabályos lépésközökkel és alakkal leírható a mutatott memóriaterületen való iteráció, így elegendő egy új `ndarray` objektum létrehozása eltérő alakkal és lépésközzel, azonban új memóriaterület lefoglalása nem szükséges. Advanced indexing esetén tetszőleges elemek hivatkozhatók, így szabályos lépésközzel nem írható le a memóriában mutatott terület, emiatt új tömb lefoglalása szükséges.

## Tömb tengelyeinek felcserélése, transzponálás

A tömb tengelyei könnyen felcserélhetők, ami ugyancsak megoldható nézettel, új tömb lefoglalása nélkül. A tengelyek felcserélésének egyik leggyakoribb használati esete a mátrixok transzponálása. Nézzük hogyan változik a lépésköz transzponálás esetén.

In [ ]:
a = np.arange(6, dtype=np.int32).reshape((2,3))
print("The 2D array 'a':\n", a)
print("  ... its shape:", a.shape)
print("  ... its strides:", a.strides)

b = np.transpose(a)
print("\nThe transposed array:\n", b)
print("  ... its shape:", b.shape)
print("  ... its strides:", b.strides)

The 2D array 'a':
 [[0 1 2]
 [3 4 5]]
  ... its shape: (2, 3)
  ... its strides: (12, 4)

The transposed array:
 [[0 3]
 [1 4]
 [2 5]]
  ... its shape: (3, 2)
  ... its strides: (4, 12)


Ahogy látjuk, a transzponálás felcserélte a tengelyek hosszát és a lépésközt is ami az indexeik növeléséhez szükséges.

## Broadcasting

A broadcasting egy tömb alakbeli egyeztetését jelenti egy másik tömbbel. Erre akkor kerül sor, amikor például elemenkénti műveleteket szeretnénk végrehajtani két tömbön és ehhez az első tömb minden elemét hozzárendeljük a másik tömb egy-egy eleméhez, azaz a két tömb alakbeli egyezése szükséges.

![alt text](https://drive.google.com/uc?export=download&id=1hCbOpGPvU84B6Jg5fpBT866w49VYcamu)

A fenti ábrán látható egy példa arra, hogyan is megy végbe a tömbök alakjának egyeztetése a broadcasting során. A második tömb eredeti alakja (3,): egyrészt egy új tengelyt kell létrehozni, majd megismételni a tömböt az új tengely mentén egyszer, így elérve a (2,3) alakot.

Az új tengely mentén az elemek ismétlése azonban csak látszólagos, ahogy azt egy broadcast-olt tömb `strides` attribútuma is megmutatja. Mivel a hétköznapi használat során a broadcasting automatikusan megy végbe a megfelelő elemenkénti művelet meghívásakor, így ahhoz, hogy jobban megvizsgálhassuk a broadcastolás eredményeképpen keletkezett nézetet, szükséges, hogy az explicit `np.broadcast_to()` műveletet használjuk, ami elvégzi az alakegyeztetést egyéb művelet végrehajtása nélkül.

In [ ]:
a = np.arange(6, dtype=np.int32).reshape((2,3))
print("The 2D array 'a':\n", a)
print("  ... its shape:", a.shape)
print("  ... its strides:", a.strides)

b = np.arange(3, dtype=np.int32)
print("\nThe 1D array 'b':\n", b)
print("  ... its shape:", b.shape)
print("  ... its strides:", b.strides)

# We will broadcast 'b' to the shape of 'a'

b = np.broadcast_to(b, a.shape)
print("\nThe 'b' array broadcast to the shape of 'a':\n", b)
print("  ... its shape:", b.shape)
print("  ... its strides:", b.strides)

The 2D array 'a':
 [[0 1 2]
 [3 4 5]]
  ... its shape: (2, 3)
  ... its strides: (12, 4)

The 1D array 'b':
 [0 1 2]
  ... its shape: (3,)
  ... its strides: (4,)

The 'b' array broadcast to the shape of 'a':
 [[0 1 2]
 [0 1 2]]
  ... its shape: (2, 3)
  ... its strides: (0, 4)


Látható, hogy a (2,3) alakra broadcast-olt 'b' tömb az új, #0 indexű tengely mentén 0 méretű lépésközzel rendelkezik. Ez pusztán azt jelenti, hogy bár látszólag a tömb alakja (2,3), valójában az csak (1,3) és a #0 indexű tengely mentén való indexelés teljességgel hatástalan, ugyanazokat az elemet fogja olvasni, függetlenül ennek az indexnek az értékétől.

## Lépésköz trükkök

A lépésköz manuális átállításával egyedi viselkedést kaphatunk, mellyel könnyen megvalósíthatók bizonyos műveletek.

Egy példa erre az ablakolt műveletvégzés. Tegyük fel, hogy egy hosszú 1 dimenziós tömbön szeretnénk egymást átfedő ablakokon átlagot számolni. Jelfeldolgozásban ez egy gyakori probléma lehet.

In [ ]:
a = np.arange(7, dtype=np.float32)
print("The 1D array 'a':\n", a)
print("  ... its shape:", a.shape)
print("  ... its strides:", a.strides)

window_len = 4

idxs = np.stack([np.arange(i, i+window_len) for i in range(a.shape[0]-window_len+1)], axis=0)
windowed_a = a[idxs]
print("\nThe windowed 'a' array:\n", windowed_a)
print("  ... its shape:", windowed_a.shape)
print("  ... its strides:", windowed_a.strides)

window_means = np.mean(windowed_a, axis=1)
print("\nThe windowed mean values:\n", window_means)

The 1D array 'a':
 [0. 1. 2. 3. 4. 5. 6.]
  ... its shape: (7,)
  ... its strides: (4,)

The windowed 'a' array:
 [[0. 1. 2. 3.]
 [1. 2. 3. 4.]
 [2. 3. 4. 5.]
 [3. 4. 5. 6.]]
  ... its shape: (4, 4)
  ... its strides: (16, 4)

The windowed mean values:
 [1.5 2.5 3.5 4.5]


A fenti egy naiv megoldás az ablakolás létrehozására és átlagolásra. Az ablakok számolásakor szükség van hozzá egy megközelítőleg _ablak_méret_*_szekvenciahossz_ elemű tömbre, ami sok helyet foglalhat a memóriában, nagy ablakméret, illetve szekvenciahossz esetén. Azonban be kell látnunk, hogy ebben a tömbben az elemek többsége redundáns, hiszen sokszor megismételjük ugyanazokat az elemeket.

Egy jobb megoldás alapötlete az, hogy az egyes tengelyek mentén való indexelés nem kell, hogy diszjunkt memóriaterületeket fedjen le, akár átfedés is lehetséges. Az eredeti _szekvenciahossz_ tengely mellett létrehozunk egy új tengelyt, ami az ablakok elemein indexel végig.

A lépésközök manuális megadásához az `numpy.lib.stride_tricks.as_strided()` függvényt fogjuk használni.

**Fontos megjegyezni, hogy a függvény nem megfelelő használata szegmentációs hibához vezethez, hiszen nem megfelelő tömb alak és lépésköz megadása esetén ki is indexelhetünk a tömb, vagy a program számára fenntartott memóriaterületről!**

A függvény alapesetben, egy csak olvasható nézetét adja vissza az eredeti tömbnek. Ha a `writeable` paramétert `True`-ra állítjuk, a nézet írható is lesz. Ezt ne tegyük meg, hiszen akár más Python objektumok memóriaterületét is felülírhatjuk vele. Ezenkívül, még "helyes" használat esetén is, az egymást átfedő szeletekből álló nézet írása megjósolhatatlan eredményű.

In [ ]:
a = np.arange(7, dtype=np.float32)
print("The 1D array 'a':\n", a)
print("  ... its shape:", a.shape)
print("  ... its strides:", a.strides)

window_len = 4

new_shape = (a.shape[0]-window_len+1, window_len)
new_strides = (a.itemsize, a.itemsize)

windowed_a = np.lib.stride_tricks.as_strided(a, shape=new_shape, strides=new_strides)
print("\nThe windowed 'a' array:\n", windowed_a)
print("  ... its shape:", windowed_a.shape)
print("  ... its strides:", windowed_a.strides)

window_means = np.mean(windowed_a, axis=1)
print("\nThe windowed mean values:\n", window_means)

The 1D array 'a':
 [0. 1. 2. 3. 4. 5. 6.]
  ... its shape: (7,)
  ... its strides: (4,)

The windowed 'a' array:
 [[0. 1. 2. 3.]
 [1. 2. 3. 4.]
 [2. 3. 4. 5.]
 [3. 4. 5. 6.]]
  ... its shape: (4, 4)
  ... its strides: (4, 4)

The windowed mean values:
 [1.5 2.5 3.5 4.5]


Az `as_strided()` segítségével kiszámolhattuk az ablakolt átlagokat úgy, hogy nem kellett lefoglalnunk új tömböt az ablakolás előállításához. Az új nézet lépésközeit úgy állítottuk be, hogy mindkét tengely mentén 1-1 elemet lépjen csak az index eggyel való növelésekor. Hiszen, egyrészt az ablakok közti lépegetéshez is, másrészt az ablakon belüli lépegetéshez is 1-1 elemet kell lépni az eredeti tömbben.

A memóriaigény töredékére esett az előző megoldáshoz képest.